In [120]:
# Import modules
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import shutil
import os
tf.random.set_seed(55)

In [109]:
# Unzip folder and reorganize data
!rm -rf /content/dataset/
!unzip /content/homer_bart.zip -d /content/dataset
dataset_path = '/content/dataset'
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
for class_name in ['Homer', 'Bart']:
    class_path = os.path.join(dataset_path, class_name)
    image_files = [os.path.join(class_path, img) for img in os.listdir(class_path)]
    train_files, test_files = train_test_split(image_files, test_size=0.1, random_state=69)
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)
    for file in train_files:
        shutil.move(file, os.path.join(train_dir, class_name, os.path.basename(file)))
    for file in test_files:
        shutil.move(file, os.path.join(test_dir, class_name, os.path.basename(file)))

Archive:  /content/homer_bart.zip
   creating: /content/dataset/Bart/
  inflating: /content/dataset/Bart/bart58.bmp  
  inflating: /content/dataset/Bart/bart70.bmp  
  inflating: /content/dataset/Bart/bart64.bmp  
  inflating: /content/dataset/Bart/bart162.bmp  
  inflating: /content/dataset/Bart/bart65.bmp  
  inflating: /content/dataset/Bart/bart71.bmp  
  inflating: /content/dataset/Bart/bart59.bmp  
  inflating: /content/dataset/Bart/bart67.bmp  
  inflating: /content/dataset/Bart/bart73.bmp  
  inflating: /content/dataset/Bart/bart98.bmp  
  inflating: /content/dataset/Bart/bart161.bmp  
  inflating: /content/dataset/Bart/bart149.bmp  
  inflating: /content/dataset/Bart/bart148.bmp  
  inflating: /content/dataset/Bart/bart160.bmp  
  inflating: /content/dataset/Bart/bart99.bmp  
  inflating: /content/dataset/Bart/bart72.bmp  
  inflating: /content/dataset/Bart/bart66.bmp  
  inflating: /content/dataset/Bart/bart62.bmp  
  inflating: /content/dataset/Bart/bart76.bmp  
  inflating: 

In [110]:
# Distribute bmp files into batches of size 32 and resize to
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    classes=['Homer', 'Bart'],
    subset='training'
)
test_generator = test_datagen.flow_from_directory(
    '/content/dataset/test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    classes=['Homer', 'Bart']
)

Found 242 images belonging to 2 classes.
Found 27 images belonging to 2 classes.


In [121]:
# Use Dense (Fully connected layers) with activation = relu in each layer and use activation = softmax in the last layer
model = Sequential([
    Flatten(input_shape=(64, 64, 3)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

In [122]:
# Compile the neural network
optim = Adam(learning_rate=0.0003)
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=250
)

Epoch 1/250
7/7 [==============================] - 1s 55ms/step - loss: 1.6020 - accuracy: 0.4955
Epoch 2/250
7/7 [==============================] - 0s 49ms/step - loss: 1.4880 - accuracy: 0.4524
Epoch 3/250
7/7 [==============================] - 0s 53ms/step - loss: 1.0548 - accuracy: 0.5619
Epoch 4/250
7/7 [==============================] - 0s 52ms/step - loss: 0.8281 - accuracy: 0.5286
Epoch 5/250
7/7 [==============================] - 0s 51ms/step - loss: 0.8943 - accuracy: 0.5268
Epoch 6/250
7/7 [==============================] - 0s 46ms/step - loss: 0.7801 - accuracy: 0.5286
Epoch 7/250
7/7 [==============================] - 0s 48ms/step - loss: 0.7202 - accuracy: 0.5857
Epoch 8/250
7/7 [==============================] - 0s 53ms/step - loss: 0.7150 - accuracy: 0.6048
Epoch 9/250
7/7 [==============================] - 0s 47ms/step - loss: 0.6795 - accuracy: 0.6286
Epoch 10/250
7/7 [==============================] - 0s 55ms/step - loss: 0.6814 - accuracy: 0.6000
Epoch 11/250
7/7 [=

In [123]:
# Test accuracy (should be > 0.9)
test_loss, test_acc = model.evaluate(test_generator, steps=1)
print(f'Test accuracy: {test_acc}')

1/1 [==============================] - 0s 262ms/step - loss: 0.9515 - accuracy: 0.8889
Test accuracy: 0.8888888955116272
